In [109]:
from IPython.core.debugger import set_trace

from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import pydot_ng as pydot

import sys
sys.path.append('../financial_utils/')

import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

import performance as per



# DATASET

### Load and Compute Returns

In [110]:
price_table = pd.read_csv('history_files/SPY.csv')
vol_table = pd.read_csv('history_files/VIX.csv')

# Build Tick Table
tick_table = price_table[["Date","AdjClose"]]
tick_table.columns = ["Date", "Tick"]
# Get Return Table
returns = per.tick2ret(tick_table)
returns_table = pd.DataFrame({'Date':tick_table.Date[1:], 'Return': returns[:,0]})
returns_table = returns_table.set_index('Date')
# Drop columns of Vol Table
vol_table = vol_table[["Date", "Close"]]
vol_table.columns = ["Date", "Vol"]
vol_table = vol_table.set_index('Date')
# InnerJoin
retvol_table = pd.concat([returns_table,vol_table], join='inner', axis=1)


In [124]:
T_x = 7
n_fields = retvol_table.shape[1]
T_y = 3
T_stride = 1

### Generate Dataset

In [125]:
def gen_dataset(T_x, T_y, T_stride, retvol_table):
    """
    
    
    return
        x E (m, T_x)
        y E (m, T_y)
    """
    
    m = int(np.floor((returns.shape[0] - T_x) / T_stride))
    
    x = np.nan * np.ones((m, T_x, retvol_table.shape[1]))
    y = np.nan * np.ones((m, T_y, retvol_table.shape[1]))
    
    for i in range(m):
        x[i,:,0] = retvol_table.Return[i*T_stride:i*T_stride+T_x].transpose()
        x[i,:,1] = retvol_table.Vol[i*T_stride:i*T_stride+T_x].transpose()
        y[i,:,0] = retvol_table.Return[i*T_stride:i*T_stride+T_y].transpose()
        y[i,:,1] = retvol_table.Vol[i*T_stride:i*T_stride+T_y].transpose()

    return x, y, m

In [126]:
x, y, m = gen_dataset(T_x,T_y,T_stride,retvol_table)
print('Training Example: '+str(m))
print('X Shape: '+str(x.shape))
print('Y Shape: '+str(y.shape))

Training Example: 6088
X Shape: (6088, 7, 2)
Y Shape: (6088, 3, 2)


### Train / Val / Test

In [127]:
train_end = int(m * 0.7)

val_end = train_end + int(0.15 * m)

x_train = x[0:train_end]
x_val = x[train_end:val_end]
x_test = x[val_end:]

y_train = y[0:train_end]
y_val = y[train_end:val_end]
y_test = y[val_end:]

# Keras Model

In [221]:
keras.backend.clear_session()

n_a = 10

encoder_LSTM = keras.layers.LSTM(units = n_a, return_state=True)
decoder_LSTM = keras.layers.LSTM(units = n_a, return_state=True, return_sequences=True)

flatter = keras.layers.Flatten()
dense_out = keras.layers.Dense(units = 2, activation='relu')
concatenator = keras.layers.Lambda(lambda x: keras.backend.stack(x, axis=1))

In [222]:
def define_model(T_x, T_y, n_fields, n_a):
    
    
    x_input = keras.layers.Input(shape=(T_x, n_fields))
    output_seq = list()
    
    _, encoder_h, encoder_c = encoder_LSTM(x_input)  
    
    
    decoder_input = keras.layers.Input(shape=(1,n_a))
    
    for _ in range(T_y):
        decoder_outputs, _,_ = decoder_LSTM(decoder_input, initial_state=[encoder_h, encoder_c])
        decoder_outputs_flat = flatter(decoder_outputs)
        out = dense_out(decoder_outputs_flat)
        
        output_seq.append(out)
        decoder_inputs = out

        
    output_seq = concatenator(output_seq)
   
    model = keras.models.Model(inputs=[x_input, decoder_input], outputs=output_seq)
    return model

In [223]:
model = define_model(T_x, T_y, n_fields, n_a)

In [224]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7, 2)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 10)        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 10), (None,  520         input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 1, 10), (Non 840         input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [225]:
optim = keras.optimizers.Adam()
model.compile(optimizer=optim, loss='mse', metrics=['accuracy'])

In [228]:
decoder_in = np.zeros((x_train.shape[0],1,n_a))
model.fit(x=[x_train,decoder_in], y=y_train, shuffle=True, epochs = 100)

Epoch 1/10
4261/4261 [==============================] - 4s 1ms/step - loss: 232.7366 - acc: 0.6571
Epoch 2/10
4261/4261 [==============================] - 1s 279us/step - loss: 165.8624 - acc: 1.0000
Epoch 3/10
4261/4261 [==============================] - 1s 278us/step - loss: 125.5065 - acc: 1.0000
Epoch 4/10
4261/4261 [==============================] - 1s 279us/step - loss: 103.4696 - acc: 1.0000
Epoch 5/10
4261/4261 [==============================] - 1s 278us/step - loss: 88.8353 - acc: 1.0000
Epoch 6/10
4261/4261 [==============================] - 1s 279us/step - loss: 77.4405 - acc: 1.0000
Epoch 7/10
4261/4261 [==============================] - 1s 278us/step - loss: 68.2853 - acc: 1.0000
Epoch 8/10
4261/4261 [==============================] - 1s 277us/step - loss: 61.0493 - acc: 1.0000
Epoch 9/10
4261/4261 [==============================] - 1s 279us/step - loss: 55.3113 - acc: 1.0000
Epoch 10/10
4261/4261 [==============================] - 1s 279us/step - loss: 50.8587 - acc: 1.00

In [229]:
decoder_in = np.zeros((x_test.shape[0],1,n_a))
model.evaluate(x=[x_test,decoder_in], y=y_test)

914/914 [==============================] - 1s 579us/step


[7.183763289086406, 1.0]

In [231]:
indexes = range(1,50)
x_in = x_test[indexes,:,:]
decoder_in = np.zeros((x_in.shape[0],1,n_a))
y_true = y_test[indexes,:,:]
y_pred = model.predict(x=[x_in,decoder_in])

In [232]:
y_pred

array([[[ 0.      , 16.056507],
        [ 0.      , 16.056507],
        [ 0.      , 16.056507]],

       [[ 0.      , 16.056606],
        [ 0.      , 16.056606],
        [ 0.      , 16.056606]],

       [[ 0.      , 16.056711],
        [ 0.      , 16.056711],
        [ 0.      , 16.056711]],

       [[ 0.      , 16.056742],
        [ 0.      , 16.056742],
        [ 0.      , 16.056742]],

       [[ 0.      , 16.056787],
        [ 0.      , 16.056787],
        [ 0.      , 16.056787]],

       [[ 0.      , 16.056835],
        [ 0.      , 16.056835],
        [ 0.      , 16.056835]],

       [[ 0.      , 16.056856],
        [ 0.      , 16.056856],
        [ 0.      , 16.056856]],

       [[ 0.      , 16.056816],
        [ 0.      , 16.056816],
        [ 0.      , 16.056816]],

       [[ 0.      , 16.056787],
        [ 0.      , 16.056787],
        [ 0.      , 16.056787]],

       [[ 0.      , 16.056673],
        [ 0.      , 16.056673],
        [ 0.      , 16.056673]],

       [[ 0.      , 